# STEP1


1.   刪除取一年中每月淨利為負的(若標準過高則再作修正)-刪除極端值
2.   所有1~3月本益比平均or當季本益比 的 describe()-刪除極端值




### 匯入套件

In [1]:
pip install jmespath

In [2]:
import requests
import json
#可從json格式中取得資料(json格式定位資料位置)
import jmespath
import pandas as pd
import time
import re
import warnings
warnings.simplefilter(action='ignore')

### 目標資訊

In [3]:
#目標股票代碼
stocklist=["1201","1203","1210","1213","1215","1216","1217","1218","1219","1220","1225","1227","1229","1231","1232","1233","1234","1235","1236","1256","1702","1737"]
#目標時間
datelist=[]
for year in range(2019,2022):
    for month in range(1,13):
        date=str(year)+str("%02d" % month)+"01"
        if date == '20210401':
          break
        else:
          datelist.append(date)

### 爬蟲(本益比資訊)

In [ ]:
def get_peratio(stock,date):
  #創建存放資訊dict
  data={"STOCK":[],"年分":[],"本益比":[]} 

  for date in datelist:
    try:
      for stock in stocklist:
        #替代成所需的股票代碼及時間
        request_url = f'https://www.twse.com.tw/exchangeReport/BWIBBU?response=json&date={date}&stockNo={stock}' 
        res = requests.get(request_url)
        resinfo = json.loads(res.text)
        #定位所需資訊
        peratio=jmespath.search('data[:][3][]',resinfo) 
        for peratioidx in range(len(peratio)):
          #若該欄位並無放置資訊或資訊非浮點數格式會出錯，因此遇到錯誤則跳過
          try:
            #將資訊填入相對應位置
            data['本益比'].append(float(peratio[peratioidx]))
            data['STOCK'].append(stock)
            data['年分'].append((re.findall(r'^2\w{3}',date)[0]))
          except TypeError:
            print(peratioidx)
            pass
        #避免過快取得網頁資訊會被封鎖ip
        time.sleep(3)
    except ValueError:
      stocklist.remove(stock)
  return data

In [ ]:
df = pd.DataFrame(get_peratio(stocklist,datelist))
#將欄位整理成以股票及年為群組，分別使用max﹑min及mean的方式
df1 = pd.concat([df.groupby(['STOCK','年分']).mean(), df.groupby(['STOCK','年分']).min(), df.groupby(['STOCK','年分']).max()], axis=1)
df1.columns = ['平均本益比','最小本益比','最大本益比']

In [4]:
# df1.to_csv('stockstep1.csv')
!gdown --id '1INiw55LxmkbWHSrq5hDPoPc-mAtQ4XEq'
df1 = pd.read_csv('stockstep1.csv', index_col=0)

Downloading...
From: https://drive.google.com/uc?id=1INiw55LxmkbWHSrq5hDPoPc-mAtQ4XEq
To: /content/stockstep1.csv
100% 2.38k/2.38k [00:00<00:00, 4.23MB/s]


# STEP 2

*   算出合理區間
*   Link：https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1201&SYEAR=2020&SSEASON=4&REPORT_ID=C





### 找出EPS

In [5]:
#1~3->ABOUT EPS

#下載所需csv
!gdown --id '1qyBFxEfEF9jDHrTvG4i5SlNWtdCr_soC'
!gdown --id '17iFuLmAL3jOocE0vMZKqMkERWQnTR85B'
!gdown --id '19bqdpigtGm1J2BsV3wJgoJP98ulWMzxv'

# 合併3個檔案
read = pd.read_csv('stock_1.csv')
read.to_csv('stockstep2_ori.csv')
for i in range(2, 4):
    read = pd.read_csv('stock_{}.csv'.format(i))
    read.to_csv('stockstep2_ori.csv', header=False, mode='a+')

Downloading...
From: https://drive.google.com/uc?id=1qyBFxEfEF9jDHrTvG4i5SlNWtdCr_soC
To: /content/stock_1.csv
100% 169k/169k [00:00<00:00, 58.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=17iFuLmAL3jOocE0vMZKqMkERWQnTR85B
To: /content/stock_2.csv
100% 169k/169k [00:00<00:00, 60.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=19bqdpigtGm1J2BsV3wJgoJP98ulWMzxv
To: /content/stock_3.csv
100% 153k/153k [00:00<00:00, 55.4MB/s]


In [6]:
df2 = pd.read_csv('stockstep2_ori.csv', index_col='Unnamed: 0')

In [7]:
#利用上方剩下的股票代碼刪除新csv中不需要的股票代碼
stocklist = [str(i) for i in list(set(df1.index.get_level_values('STOCK')))]
df2 = df2[df2['公司代號'].isin(stocklist)]

In [8]:
df2 = df2[[x not in list(set(df2[df2['基本每股盈餘（元）']<0]['公司代號'].tolist())) for x in df2['公司代號']]]

In [9]:
#資料為兩年即一記得每股盈餘，因此加總/2.25
df2 = df2.groupby(['公司代號']).sum()/2.25
#刪除不需要的欄位
df2 = df2.drop(df2.columns[:-1], axis=1)
#重新命名欄位名稱
df2 = df2.rename(columns={'公司代號': 'STOCK','基本每股盈餘（元）': 'EPS'})

In [10]:
df2.index.names = ['STOCK']

### 算出合理區間

In [11]:
#以STOCK欄位合併兩張資料表
df = pd.merge(df1,df2,on='STOCK')
df = pd.concat([df.groupby(['STOCK']).min().filter(['STOCK','最小本益比'], axis=1), df.groupby(['STOCK']).max().filter(['STOCK','最大本益比'], axis=1), df.groupby(['STOCK']).mean().filter(['STOCK','平均本益比','EPS'], axis=1)], axis=1)

In [12]:
#算出股價區間
df['便宜股價'] = df['最小本益比']*df['EPS']
df['安全股價'] = ((df['最小本益比']+df['平均本益比'])/2)*df['EPS']
df['合理股價'] = df['平均本益比']*df['EPS']
df['觀察股價'] = ((df['最大本益比']+df['平均本益比'])/2)*df['EPS']
df['昂貴股價'] = df['最大本益比']*df['EPS']

In [13]:
#取小數點後第5位
df = df.round(5)

In [14]:
df.filter(df.columns.tolist()[4:9], axis=1).to_csv('stockstep2.csv', encoding='big5')
!gdown --id '1sARbyWVvtU_oTujMYAB58B4_9XrdnFjM'
df2 = pd.read_csv('stockstep2.csv', index_col=0, encoding='big5')

Downloading...
From: https://drive.google.com/uc?id=1sARbyWVvtU_oTujMYAB58B4_9XrdnFjM
To: /content/stockstep2.csv
100% 1.03k/1.03k [00:00<00:00, 4.31MB/s]


In [15]:
!gdown --id '1vD3StawgkocwC8YSXT9CVmq7WDV0UixV'
usecols = ['證券代號','收盤價']
read = pd.read_csv('stock_12.csv', index_col=0, header=2, usecols=usecols)
# stocklist = [str(i) for i in list(set(df2.index.get_level_values('STOCK')))]
df3_4 = read[read.index.isin(stocklist)]
df3_4.index.names = ['STOCK']
df3_4.index = df3_4.index.astype(int)
df3_4 = pd.concat([df2, df3_4], axis=1)
df3_4['SCORE_4']= (df3_4['合理股價'] - df3_4['收盤價']).rank()*2

Downloading...
From: https://drive.google.com/uc?id=1vD3StawgkocwC8YSXT9CVmq7WDV0UixV
To: /content/stock_12.csv
100% 2.85k/2.85k [00:00<00:00, 4.39MB/s]


In [16]:
#4~5->ABOUT EPS

#下載所需csv
!gdown --id '1-PIx_K_r8-akJYtVYnJ6fo8WQ_Qv1r-H'
!gdown --id '1bh4q9bEDFVFxEId50ovllExZmtl17EdW'
!gdown --id '1Bo4uoGEMM_MXKm87GPfhwA2bk4O5BjVy'

usecols = ['公司代號','毛利率(%)(營業毛利)/(營業收入)','營業利益率(%)(營業利益)/(營業收入)','稅後純益率(%)(稅後純益)/(營業收入)']

# 合併3個檔案
read = pd.read_csv('stock_4.csv', index_col=0, usecols=usecols)
for i in range(5,7):
    read1 = pd.read_csv('stock_{}.csv'.format(i), index_col=0, usecols=usecols)
    read = pd.merge(read,read1,on='公司代號')


stocklist = [str(i) for i in list(set(df2.index.get_level_values('STOCK')))]
df3_1 = read[read.index.isin(stocklist)]
df3_1.index.names = ['STOCK']
df3_1['SCORE_1']=0

Downloading...
From: https://drive.google.com/uc?id=1-PIx_K_r8-akJYtVYnJ6fo8WQ_Qv1r-H
To: /content/stock_4.csv
100% 40.5k/40.5k [00:00<00:00, 14.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bh4q9bEDFVFxEId50ovllExZmtl17EdW
To: /content/stock_5.csv
100% 41.1k/41.1k [00:00<00:00, 15.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Bo4uoGEMM_MXKm87GPfhwA2bk4O5BjVy
To: /content/stock_6.csv
100% 40.7k/40.7k [00:00<00:00, 14.7MB/s]


In [17]:
!gdown --id '13krszbw-j3a3lEOsnrohd62ErYa1KOSn'
!gdown --id '1OoYey4KXJaGPnYsGyWAh8R0ME5YscZ25'
!gdown --id '1NDmk6RNxqK9F7ea9mD4hfYpEZwicTUB5'

usecols = ['證  券  名  稱','成  交  量']

# 合併3個檔案
read = pd.read_csv('stock_7.csv', header=1, index_col=0, usecols=usecols)
read = read.iloc[16:38]
for i in range(8,10):
    read1 = pd.read_csv('stock_{}.csv'.format(i), header=1, index_col=0, usecols=usecols)
    read1 = read1.iloc[16:38]
    read = pd.merge(read,read1,on='證  券  名  稱')

read.index = read.index.to_series().apply(lambda x:re.findall(r'^1\w{3}',x)[0])
# stocklist = [str(i) for i in list(set(df2.index.get_level_values('STOCK')))]
df3_2 = read[read.index.isin(stocklist)]
df3_2.index.names = ['STOCK']
df3_2.index = df3_2.index.astype(int)
df3_2['SCORE_2'] = 0
for idx in df3_2.columns[:-1]:
  df3_2[idx] = df3_2[idx].apply(lambda x:x.replace(",", ""))

Downloading...
From: https://drive.google.com/uc?id=13krszbw-j3a3lEOsnrohd62ErYa1KOSn
To: /content/stock_7.csv
100% 1.88M/1.88M [00:00<00:00, 60.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OoYey4KXJaGPnYsGyWAh8R0ME5YscZ25
To: /content/stock_8.csv
100% 1.85M/1.85M [00:00<00:00, 57.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NDmk6RNxqK9F7ea9mD4hfYpEZwicTUB5
To: /content/stock_9.csv
100% 1.94M/1.94M [00:00<00:00, 60.5MB/s]


In [18]:
!gdown --id '1cCu1PyZHAQ-U7aWBzHoMCX_czuvaigOo'
!gdown --id '1200qFzf6D2hf1fBCLhvsEhsDt86AALFY'
usecols = ['公司代號','財務結構-負債佔資產比率(%)','償債能力-速動比率(%)','經營能力-應收款項週轉率(次)','經營能力-存貨週轉率(次)','經營能力-總資產週轉率(次)']
 
# 合併2個檔案
read = pd.read_csv('stock_10.csv', index_col=0, usecols=usecols)
read1 = pd.read_csv('stock_11.csv', index_col=0, usecols=usecols)
read = pd.merge(read,read1,on='公司代號')
 
# stocklist = [str(i) for i in list(set(df2.index.get_level_values('STOCK')))]
df3_3 = read[read.index.isin(stocklist)]
df3_3.index.names = ['STOCK']
df3_3['SCORE_3'] = 0

Downloading...
From: https://drive.google.com/uc?id=1cCu1PyZHAQ-U7aWBzHoMCX_czuvaigOo
To: /content/stock_10.csv
100% 118k/118k [00:00<00:00, 41.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1200qFzf6D2hf1fBCLhvsEhsDt86AALFY
To: /content/stock_11.csv
100% 118k/118k [00:00<00:00, 35.2MB/s]


In [19]:
for  idx in df3_1.columns:
  df3_1[idx] = pd.to_numeric(df3_1[idx])
for  idx in df3_2.columns:
  df3_2[idx] = pd.to_numeric(df3_2[idx])
for  idx in df3_3.columns:
  df3_3[idx] = pd.to_numeric(df3_3[idx])

In [20]:
# def col1(df):
for idx in range(len(df3_2)):
  if (df3_2['成  交  量_x'].iloc[idx]+df3_2['成  交  量_y'].iloc[idx]+df3_2['成  交  量'].iloc[idx])/3>1000000:
    df3_2['SCORE_2'].iloc[idx] = df3_2['SCORE_2'].iloc[idx]+10
    # return df
# def col2(df):
for idx in range(len(df3_3)):
  if df3_3['財務結構-負債佔資產比率(%)_y'].iloc[idx]<50:
    df3_3['SCORE_3'].iloc[idx]= df3_3['SCORE_3'].iloc[idx]+10
    # return df
# def col3(df):
for idx in range(len(df3_3)):
  if df3_3['償債能力-速動比率(%)_y'].iloc[idx]>80:
    df3_3['SCORE_3'].iloc[idx] = df3_3['SCORE_3'].iloc[idx]+10
    # return df
# def col4(df):
for idx in range(len(df3_3)):
  if df3_3['經營能力-應收款項週轉率(次)_y'].iloc[idx]-df3_3['經營能力-應收款項週轉率(次)_x'].iloc[idx]>0:
    df3_3['SCORE_3'].iloc[idx] = df3_3['SCORE_3'].iloc[idx]+5
    # return df
# def col5(df):
for idx in range(len(df3_3)):
  if df3_3['經營能力-存貨週轉率(次)_y'].iloc[idx]-df3_3['經營能力-存貨週轉率(次)_x'].iloc[idx]>0:
    df3_3['SCORE_3'].iloc[idx] = df3_3['SCORE_3'].iloc[idx]+5
    # return df
# def col6(df):
for idx in range(len(df3_3)):
  if df3_3['經營能力-總資產週轉率(次)_y'].iloc[idx]>0.9:
    df3_3['SCORE_3'].iloc[idx] = df3_3['SCORE_3'].iloc[idx]+5
    # return df
# def col7(df):
for idx in range(len(df3_1)):
  if df3_1['毛利率(%)(營業毛利)/(營業收入)'].iloc[idx]-df3_1['毛利率(%)(營業毛利)/(營業收入)_y'].iloc[idx]>0:
    df3_1['SCORE_1'].iloc[idx] = df3_1['SCORE_1'].iloc[idx]+10
    # return df
# def col8(df):
for idx in range(len(df3_1)):
  if df3_1['營業利益率(%)(營業利益)/(營業收入)'].iloc[idx]-df3_1['毛利率(%)(營業毛利)/(營業收入)'].iloc[idx]<20:
    if df3_1['營業利益率(%)(營業利益)/(營業收入)'].iloc[idx]-df3_1['營業利益率(%)(營業利益)/(營業收入)_x'].iloc[idx]>0:
        df3_1['SCORE_1'].iloc[idx] = df3_1['SCORE_1'].iloc[idx]+10
    # return df
# def col9(df):
for idx in range(len(df3_1)):
  if df3_1['稅後純益率(%)(稅後純益)/(營業收入)'].iloc[idx]>0:
    if df3_1['稅後純益率(%)(稅後純益)/(營業收入)_x'].iloc[idx]>0:
      if df3_1['稅後純益率(%)(稅後純益)/(營業收入)'].iloc[idx]-df3_1['稅後純益率(%)(稅後純益)/(營業收入)_x'].iloc[idx]>0.01:
          df3_1['SCORE_1'].iloc[idx] = df3_1['SCORE_1'].iloc[idx]+10
    # return df

In [21]:
# df3_1 = col9(col8(col7(df3_1)))
# df3_2 = col1(df3_2)
# df3_3 = col6(col5(col4(col3(col2(df3_3)))))

In [22]:
df = pd.concat([df3_1.filter(['STOCK','SCORE_1'], axis=1), df3_2.filter(['SCORE_2'], axis=1), df3_3.filter(['SCORE_3'], axis=1), df3_4.filter(['SCORE_4'], axis=1)], axis=1).fillna(0)

In [23]:
df['FINAL_SCORE'] = df.sum(axis = 1)
df['FINAL_RANK'] = df['FINAL_SCORE'].rank()

In [24]:
df

,SCORE_1,SCORE_2,SCORE_3,SCORE_4,FINAL_SCORE,FINAL_RANK
STOCK,,,,,,
1201,30.0,10.0,5.0,36.0,81.0,16.0
1203,20.0,10.0,20.0,6.0,56.0,6.0
1210,10.0,10.0,25.0,8.0,53.0,3.0
1215,20.0,10.0,15.0,18.0,63.0,10.0
1216,10.0,10.0,10.0,20.0,50.0,2.0
1217,10.0,10.0,15.0,32.0,67.0,12.0
1218,10.0,10.0,25.0,10.0,55.0,5.0
1219,20.0,10.0,25.0,38.0,93.0,19.0
1220,30.0,10.0,20.0,26.0,86.0,17.0


In [25]:
df.to_csv('stockstep3.csv', encoding='big5')